Setup up imports

In [36]:
import os
import pandas as pd
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re
import matplotlib.pyplot as plt
import requests
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# load client credentials using .env file
# SPOTIPY_CLIENT_ID=YOUR_CLIENT_ID
# SPOTIPY_CLIENT_SECRET=YOUR_CLIENT_SECRET
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Lets get started by preprocessing our data
First we need to load the data from the JSON files given

In [37]:
DATA_DIR = 'spotify_million_playlist_dataset/data/'

playlists = []
for file in sorted(os.scandir(DATA_DIR), key=lambda e: e.name):
    print("processing slice: " + str(file.name))
    data = json.load(open(file.path))
    playlists.append(pd.DataFrame(data['playlists']))
    break

processing slice: mpd.slice.0-999.json


Let's clean up our array of playlists a little bit by combining them into a pandas DataFrame

In [38]:
playlists_frame = pd.concat(playlists)
playlists_frame.head()

               name collaborative  pid  modified_at  num_tracks  num_albums  \
0        Throwbacks         false    0   1493424000          52          47   
1  Awesome Playlist         false    1   1506556800          39          23   
2           korean          false    2   1505692800          64          51   
3               mat         false    3   1501027200         126         107   
4               90s         false    4   1401667200          17          16   

   num_followers                                             tracks  \
0              1  [{'pos': 0, 'artist_name': 'Missy Elliott', 't...   
1              1  [{'pos': 0, 'artist_name': 'Survivor', 'track_...   
2              1  [{'pos': 0, 'artist_name': 'Hoody', 'track_uri...   
3              1  [{'pos': 0, 'artist_name': 'Camille Saint-Saën...   
4              2  [{'pos': 0, 'artist_name': 'The Smashing Pumpk...   

   num_edits  duration_ms  num_artists description  
0          6     11532414           37       

### Now we can begin our analysis (sort of):

We'll start by querying the Spotify API with the Spotipy package to get some more details
on the songs that are contained within each playlist. Spotify calculates various data for each song
such as the time signature, tempo, timbre, etc.

In [39]:
#authorize our API session with credentials stored in the environment variables

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

Spotify provides us with two different types of song data. One is called the [audio features](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/),
and the other is the [audio analysis](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis).
Essentially, the features are Spotify's interpretation of the audio analysis, it has higher-level attributes
like the 'danceability' and 'liveness' of a song. The audio analysis is every single piece of data
that Spotify was able to calculate from the songs sound signature.

Audio features sounds like it might be a little easier to handle, so we'll define a function to query
that data first.

In [40]:
def processSongFeatures(playlist, sp):
    songs = playlist['tracks']

    # get features for all songs
    song_features = []
    song_ids = []
    for song in songs:
        # get song id
        song_id = re.sub('spotify:track:', '', song['track_uri'])
        song_ids.append(song_id)
        print('processing: ' + song['track_name'])
        features = sp.audio_features(song_id)[0]
        song_features.append(features)

    # convert features into dataframe by song id
    features_by_id = pd.DataFrame(song_features, index=song_ids)
    features_by_id.index.name = 'song_id'
    features_by_id.head()

    # export data
    """export_dir = playlist['name'] + '-' + playlist['id']
    if not os.path.isdir(export_dir):
        os.mkdir(export_dir)

    features_by_id.to_csv(export_dir + '/features.csv')"""

Next we can work on handling the audio analysis data, this is a little bit more complicated. We'll start
by extracting the list of songs in each playlist from our dataframe of playlists. Using Spotify,
we query the API for the audio analysis for each song and create a map entry for it so we can associate
each song with its analysis.

In [41]:
# select just the first playlist (for testing purposes)
playlist = playlists_frame.iloc[0]
playlist

MAX_RETRIES = 5
# get the songs from the first playlist
songs = playlist['tracks']

analyses = {}
# collect analyses of songs, dict of (song id : analysis)
for i, song in enumerate(songs):
    song_id = re.sub('spotify:track:', '', song['track_uri'])
    # sometimes the api request times out, we'll skip the song if it exceeds the max retries
    for request_attempt in range(MAX_RETRIES):
        print('analyzing: ' + song['track_name'])
        try:
            a = sp.audio_analysis(track_id=song_id)
        except requests.exceptions.ReadTimeout as rto:
            print('request to Spotify timed out for: ' + song['track_name'])
        else:
            break
    else:
        continue
    # remove some useless crap
    a.pop('meta')
    analyses[song_id] = a

name                                                    Throwbacks
collaborative                                                false
pid                                                              0
modified_at                                             1493424000
num_tracks                                                      52
num_albums                                                      47
num_followers                                                    1
tracks           [{'pos': 0, 'artist_name': 'Missy Elliott', 't...
num_edits                                                        6
duration_ms                                               11532414
num_artists                                                     37
description                                                    NaN
Name: 0, dtype: object
analyzing: Lose Control (feat. Ciara & Fat Man Scoop)
analyzing: Toxic
analyzing: Crazy In Love
analyzing: Rock Your Body
analyzing: It Wasn't Me
analyzing: Yeah!
analyzing:

Before moving on, lets check out what an analysis looks like so we can decide how we want to
structure our data.

In [42]:
# remove an (id, analysis) pair from the dictionary
analysis_pair = analyses.popitem()

# get the JSON response
json = analysis_pair[1]
# because the analysis is a JSON response (dictionary), we can access it like this
[print(str(category) + str(value)[0:100]) for category, value in json.items()]

# don't forget to add the pair back
analyses[analysis_pair[0]] = analysis_pair[1]

track{'num_samples': 4853940, 'duration': 220.13333, 'sample_md5': '', 'offset_seconds': 0, 'window_secon
bars[{'start': 0.25319, 'duration': 1.43706, 'confidence': 0.471}, {'start': 1.69025, 'duration': 1.4400
beats[{'start': 0.25319, 'duration': 0.36878, 'confidence': 0.911}, {'start': 0.62197, 'duration': 0.3587
sections[{'start': 0.0, 'duration': 6.73147, 'confidence': 1.0, 'loudness': -10.743, 'tempo': 167.388, 'temp
segments[{'start': 0.0, 'duration': 0.19773, 'confidence': 0.0, 'loudness_start': -60.0, 'loudness_max_time'
tatums[{'start': 0.25319, 'duration': 0.18439, 'confidence': 0.911}, {'start': 0.43758, 'duration': 0.1843


You know that's kind of messy, here's the [API reference](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/)
for the structure of an audio analysis.

TL;DR, the response contains categories like `track` (special one),
`bars`, `beats`, etc. `Track` is special because it contains more general data such as the `duration`
and `time_signature`, these headers aren't in the other categories. The categories `bars`, `beats`,
and `tatums`, are all composed of [time interval objects](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/#time-interval-object)
which are just dictionaries with the start, end and how confident Spotify is that the data is right.

This brings us to `sections` and `segments`. `sections` is composed of [section objects](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/#section-object)
which are basically the same as the other categories, just with more dictionary keys-value pairs.
`segments` is where it gets more interesting (and where I got stuck). Again, `segments` is composed
of another object (aptly named) [segment objects](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/#section-object).
This is another dictionary with a caveat, some of the values are actually arrays, and not just an integer
as before. At first, I thought that this nesting would be a problem, but I found that pandas takes care
of this on its own, parsing the array as a pandas series.


#### Moving on...
Now we have a dictionary mapping a song to its analysis JSON response. Let's loop through everything
and parse it into a pandas Dataframe using a handy function called `json_normalize`. We're also gonna
save the song ids for later to make creating our final dataframe a little easier.

The catch here is that we're gonna split everything up by each analysis category, and then by song id.

In [43]:
ids = []
category_frames = {}
# iterate through all songs and their analyses
for song_id, a in analyses.items():
    print('processing song: ' + song_id)
    ids.append(song_id)
    # collect the analysis data for each category in the songs' analysis
    for category, vals in a.items():
        print('processing category: ' + category)
        norm = pd.json_normalize(vals)
        # val_list is a list of dataframes
        val_list = category_frames.get(category, [])
        # add the new analysis data for the category
        val_list.append(norm)
        category_frames[category] = val_list

[print(name) for name, frame_list in category_frames.items()]

processing song: 0UaMYEvWZi0ZqiDOoHU3YI
processing category: track
processing category: bars
processing category: beats
processing category: sections
processing category: segments
processing category: tatums
processing song: 6I9VzXrHxO9rA9A5euc8Ak
processing category: track
processing category: bars
processing category: beats
processing category: sections
processing category: segments
processing category: tatums
processing song: 0WqIKmW4BTrj3eJFmnCKMv
processing category: track
processing category: bars
processing category: beats
processing category: sections
processing category: segments
processing category: tatums
processing song: 1AWQoqb9bSvzTjaLralEkT
processing category: track
processing category: bars
processing category: beats
processing category: sections
processing category: segments
processing category: tatums
processing song: 1lzr43nnXAijIGYnCT8M8H
processing category: track
processing category: bars
processing category: beats
processing category: sections
processing categor

[None, None, None, None, None, None]

Finally, we're getting a little closer to our final, 2 leveled multi-index pandas Dataframe.
We're going to use our `ids` array we created earlier to concatenate dataframes by their song ids.
This is the second, inner-level multi-index.

In [44]:
category_tables = {}
# concatenate the list of frames in each category to make tables based on song id
for cat_name, frame_list in category_frames.items():
    category_tables[cat_name] = pd.concat(frame_list, keys=ids)

Then we can concatenate again based off of the category names to create our outermost multi-index.

In [45]:
# combine the dictionary of categories and their lists of songs
category_tables_frame = pd.concat(category_tables.values(), keys=category_tables.keys())

Our final pandas dataframe has a structure like this:

| level=0   |     level=1           |     start    | ... | timbre            |
| :-: | :-: | :-: | :-: | --- |
| segments  | 3ELm3eyRhR4tF1ncqzMQEV|   252.15601  | ... | [pandas.Series]   |
|    -       |           -            |   258.15601  | ... | [pandas.Series]   |
|     -      |           -            |   259.15601  | ... | [pandas.Series]   |
|      -     |           -            |   261.15601  | ... | [pandas.Series]   |
|        -   | 2pbxqEYiXJTvFsybGGgSAi|   237.02356  | ... | [pandas.Series]   |
|       -    |           -            |   248.15601  | ... | [pandas.Series]   |
|      -     |           -            |   249.15601  | ... | [pandas.Series]   |
|      -     |           -            |   251.15601  | ... | [pandas.Series]   |
| tatums  | 3ELm3eyRhR4tF1ncqzMQEV|   252.15601  | ... | NaN   |
|    -       |           -            |   258.15601  | ... | NaN   |
|     -      |           -            |   259.15601  | ... | NaN   |
|      -     |           -            |   261.15601  | ... | NaN   |
|        -   | 2pbxqEYiXJTvFsybGGgSAi|   237.02356  | ... | NaN   |
|       -    |           -            |   248.15601  | ... | NaN  |
|      -     |           -            |   249.15601  | ... | NaN   |
|      -     |           -            |   251.15601  | ... | NaN   |

Let's combine all of that into a function that returns our completed dataframe.

In [46]:
# this function creates a multi-index dataframe and exports it to csv
# - level 1 is the category of the analysis
# - level 2 is the song id for the data

def processAnalyses(playlist, sp, path='playlist_frames/'):
    # get the songs from the first playlist
    songs = playlist['tracks']

    analyses = {}
    # collect analyses of songs, dict of (song id : analysis)
    for i, song in enumerate(songs):
        song_id = re.sub('spotify:track:', '', song['track_uri'])
        # sometimes the api request times out, we'll skip the song if it exceeds the max retries
        for request_attempt in range(MAX_RETRIES):
            print('analyzing: ' + song['track_name'])
            try:
                a = sp.audio_analysis(track_id=song_id)
            except requests.exceptions.ReadTimeout as rto:
                print('request to Spotify timed out for: ' + song['track_name'])
            else:
                break
        else:
            continue
        # remove some useless crap
        a.pop('meta')
        analyses[song_id] = a

    ids = []
    category_frames = {}
    # iterate through all songs and their analyses
    for song_id, a in analyses.items():
        print('processing song: ' + song_id)
        ids.append(song_id)
        # collect the analysis data for each category in the songs' analysis
        for category, vals in a.items():
            print('processing category: ' + category)
            norm = pd.json_normalize(vals)
            # val_list is a list of dataframes
            val_list = category_frames.get(category, [])
            # add the new analysis data for the category
            val_list.append(norm)
            category_frames[category] = val_list

    category_tables = {}
    # concatenate the list of frames in each category to make tables based on song id
    for cat_name, frame_list in category_frames.items():
        category_tables[cat_name] = pd.concat(frame_list, keys=ids)

    # combine the dictionary of categories and their lists of songs
    category_tables_frame = pd.concat(category_tables.values(), keys=category_tables.keys())

    # export the data to csv
    file_name = playlist['pid']
    category_tables_frame.to_csv(path + str(file_name) + '.csv')

    return category_tables_frame

    # enable to export individual tables for each category, grouped by song
    # not really needed if you use groupby
    """
    export_dir = playlist['name'] + '-' + playlist['pid']
    if not os.path.isdir(export_dir):
        os.mkdir(export_dir)

    print(category_tables.keys())
    for category, table in category_tables.items():
        print(category)
        idx = pd.IndexSlice
        table.to_csv(export_dir + '/' + category + '.csv')"""

Just to check that everything's working, we'll call our brand new function

In [89]:
playlist_analysis = processAnalyses(playlist, sp)
playlist_analysis

analyzing: Lose Control (feat. Ciara & Fat Man Scoop)
analyzing: Toxic
analyzing: Crazy In Love
request to Spotify timed out for: Crazy In Love
analyzing: Crazy In Love
analyzing: Rock Your Body
analyzing: It Wasn't Me
analyzing: Yeah!
analyzing: My Boo
analyzing: Buttons
analyzing: Say My Name
analyzing: Hey Ya! - Radio Mix / Club Mix
analyzing: Promiscuous
analyzing: Right Where You Want Me - Radio Edit Version
analyzing: Beautiful Soul
analyzing: Leavin'
analyzing: Me & U
analyzing: Ice Box
analyzing: Sk8er Boi
analyzing: Run It!
analyzing: Check On It - feat. Bun B and Slim Thug
analyzing: Jumpin', Jumpin'
analyzing: Soak Up The Sun
analyzing: Where Is The Love?
analyzing: Stacy's Mom
analyzing: Just The Girl
analyzing: Yo (Excuse Me Miss)
analyzing: Year 3000
analyzing: Lip Gloss
analyzing: Everytime We Touch - Radio Edit
analyzing: Whatcha Say
analyzing: Miss Independent
analyzing: Party In The U.S.A.
analyzing: The Great Escape
analyzing: Replay
analyzing: Forever
analyzing: You

num_samples   duration sample_md5  \
track  0UaMYEvWZi0ZqiDOoHU3YI 0       5002341.0  226.86354              
       6I9VzXrHxO9rA9A5euc8Ak 0       4383540.0  198.80000              
       0WqIKmW4BTrj3eJFmnCKMv 0       5202330.0  235.93333              
       1AWQoqb9bSvzTjaLralEkT 0       5893230.0  267.26666              
       1lzr43nnXAijIGYnCT8M8H 0       5018580.0  227.60000              
...                                         ...        ...        ...   
tatums 1b7vg5T9YKR3NNqXfBYRF7 1209          NaN    0.18137        NaN   
                              1210          NaN    0.18133        NaN   
                              1211          NaN    0.18133        NaN   
                              1212          NaN    0.18133        NaN   
                              1213          NaN    0.18133        NaN   

                                    offset_seconds  window_seconds  \
track  0UaMYEvWZi0ZqiDOoHU3YI 0                0.0             0.0   
       6I9VzXrHxO9rA9A5euc8Ak 0                0.0             0.0   
       0WqIKmW4BTrj3eJFmnCKMv 0                0.0             0.0   
       1AWQoqb9bSvzTjaLralEkT 0                0.0             0.0   
       1lzr43nnXAijIGYnCT8M8H 0                0.0             0.0   
...                                            ...             ...   
tatums 1b7vg5T9YKR3NNqXfBYRF7 1209             NaN             NaN   
                              1210             NaN             NaN   
                              1211             NaN             NaN   
                              1212             NaN             NaN   
                              1213             NaN             NaN   

                                    analysis_sample_rate  analysis_channels  \
track  0UaMYEvWZi0ZqiDOoHU3YI 0                  22050.0                1.0   
       6I9VzXrHxO9rA9A5euc8Ak 0                  22050.0                1.0   
       0WqIKmW4BTrj3eJFmnCKMv 0                  22050.0                1.0   
       1AWQoqb9bSvzTjaLralEkT 0                  22050.0                1.0   
       1lzr43nnXAijIGYnCT8M8H 0                  22050.0                1.0   
...                                                  ...                ...   
tatums 1b7vg5T9YKR3NNqXfBYRF7 1209                   NaN                NaN   
                              1210                   NaN                NaN   
                              1211                   NaN                NaN   
                              1212                   NaN                NaN   
                              1213                   NaN                NaN   

                                    end_of_fade_in  start_of_fade_out  \
track  0UaMYEvWZi0ZqiDOoHU3YI 0            0.21469          226.86354   
       6I9VzXrHxO9rA9A5euc8Ak 0            0.22617          198.80000   
       0WqIKmW4BTrj3eJFmnCKMv 0            0.37664          226.50485   
       1AWQoqb9bSvzTjaLralEkT 0            0.25011          255.08862   
       1lzr43nnXAijIGYnCT8M8H 0            0.00000          223.10313   
...                                            ...                ...   
tatums 1b7vg5T9YKR3NNqXfBYRF7 1209             NaN                NaN   
                              1210             NaN                NaN   
                              1211             NaN                NaN   
                              1212             NaN                NaN   
                              1213             NaN                NaN   

                                    loudness  ...  \
track  0UaMYEvWZi0ZqiDOoHU3YI 0       -7.105  ...   
       6I9VzXrHxO9rA9A5euc8Ak 0       -3.914  ...   
       0WqIKmW4BTrj3eJFmnCKMv 0       -6.583  ...   
       1AWQoqb9bSvzTjaLralEkT 0       -6.055  ...   
       1lzr43nnXAijIGYnCT8M8H 0       -4.596  ...   
...                                      ...  ...   
tatums 1b7vg5T9YKR3NNqXfBYRF7 1209       NaN  ...   
                              1210       NaN  ...   
               

The multi-index dataframe is very powerful and allows us to do easily group by song id or analysis category.
For example, we'll use it select the `sections` category of all the songs.

In [78]:
# access just the sections category of our song analysis
sections = playlist_analysis.loc['sections']

# drop the empty columns
sections = sections.dropna(axis=1,how='all')
sections

duration  loudness    tempo  tempo_confidence  \
0UaMYEvWZi0ZqiDOoHU3YI 0   4.55718   -12.258  125.983             0.870   
                       1  15.28787    -7.235  125.538             0.795   
                       2  53.10138    -5.956  125.438             0.656   
                       3  10.03679    -7.260  126.548             0.549   
                       4  26.30849    -6.338  125.526             0.761   
...                            ...       ...      ...               ...   
1b7vg5T9YKR3NNqXfBYRF7 5  49.39999    -3.493  166.894             0.185   
                       6  16.54521    -5.271  166.999             0.314   
                       7  35.00812    -2.961  166.975             0.129   
                       8  14.37013    -3.772  167.414             0.098   
                       9   9.50738   -20.928  166.865             0.168   

                          time_signature  time_signature_confidence  key  \
0UaMYEvWZi0ZqiDOoHU3YI 0             4.0                        1.0  2.0   
                       1             4.0                        1.0  7.0   
                       2             4.0                        1.0  4.0   
                       3             4.0                        1.0  7.0   
                       4             4.0                        1.0  0.0   
...                                  ...                        ...  ...   
1b7vg5T9YKR3NNqXfBYRF7 5             4.0                        1.0  7.0   
                       6             4.0                        1.0  2.0   
                       7             4.0                        1.0  7.0   
                       8             4.0                        1.0  2.0   
                       9             4.0                        1.0  2.0   

                          key_confidence  mode  mode_confidence      start  \
0UaMYEvWZi0ZqiDOoHU3YI 0           0.237   1.0            0.293    0.00000   
                       1           0.563   1.0            0.695    4.55718   
                       2           0.837   0.0            0.598   19.84505   
                       3           0.011   1.0            0.421   72.94643   
                       4           0.536   1.0            0.447   82.98323   
...                                  ...   ...              ...        ...   
1b7vg5T9YKR3NNqXfBYRF7 5           0.394   1.0            0.626   95.30251   
                       6           0.629   1.0            0.601  144.70248   
                       7           0.702   1.0            0.650  161.24770   
                       8           0.026   1.0            0.352  196.25581   
                       9           0.000   1.0            0.000  210.62596   

                          confidence  
0UaMYEvWZi0ZqiDOoHU3YI 0       1.000  
                       1       0.668  
                       2       0.117  
                       3       0.205  
                       4       0.092  
...                              ...  
1b7vg5T9YKR3NNqXfBYRF7 5       0.859  
                       6       1.000  
                       7       0.487  
                       8       0.124  
                       9       1.000  

[499 rows x 12 columns]

Next, we'll select all analysis categories for a specific song using its ID.

In [86]:
# group the dataframe by song id
group_by_song_id = playlist_analysis.groupby(level=1)

# select all rows that have the group 0UaMYEvWZi0ZqiDOoHU3YI
group_by_song_id.get_group('0UaMYEvWZi0ZqiDOoHU3YI')

num_samples  duration sample_md5  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0            NaN   1.95001        NaN   
                              1            NaN   1.89266        NaN   
                              2            NaN   1.89015        NaN   
                              3            NaN   1.91268        NaN   
                              4            NaN   1.91566        NaN   
...                                        ...       ...        ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941          NaN   0.23942        NaN   
                              942          NaN   0.24014        NaN   
                              943          NaN   0.24014        NaN   
                              944          NaN   0.24014        NaN   
                              945          NaN   0.24014        NaN   

                                   offset_seconds  window_seconds  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0               NaN             NaN   
                              1               NaN             NaN   
                              2               NaN             NaN   
                              3               NaN             NaN   
                              4               NaN             NaN   
...                                           ...             ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941             NaN             NaN   
                              942             NaN             NaN   
                              943             NaN             NaN   
                              944             NaN             NaN   
                              945             NaN             NaN   

                                   analysis_sample_rate  analysis_channels  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0                     NaN                NaN   
                              1                     NaN                NaN   
                              2                     NaN                NaN   
                              3                     NaN                NaN   
                              4                     NaN                NaN   
...                                                 ...                ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941                   NaN                NaN   
                              942                   NaN                NaN   
                              943                   NaN                NaN   
                              944                   NaN                NaN   
                              945                   NaN                NaN   

                                   end_of_fade_in  start_of_fade_out  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0               NaN                NaN   
                              1               NaN                NaN   
                              2               NaN                NaN   
                              3               NaN                NaN   
                              4               NaN                NaN   
...                                           ...                ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941             NaN                NaN   
                              942             NaN                NaN   
                              943             NaN                NaN   
                              944             NaN                NaN   
                              945             NaN                NaN   

                                   loudness  ...  rhythmstring  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0         NaN  ...           NaN   
                              1         NaN  ...           NaN   
                              2         NaN  ...           NaN   
                              3         NaN  ...           NaN   
                              4         NaN  ...           NaN   
...                                     ...  ...           ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941       NaN  ...           NaN   
                       

Now, we can see that the `tracks` analysis category is way different than the others. We can
again use pandas to drop that group and clean up our output a little better for viewing.
Notice how each category still has tons of completely empty columns, this is because
each category doesn't have all of the same columns as the others.

In [91]:
# remove the tracks group
without_tracks = playlist_analysis.drop(index='track')

# group by song id
group_by_song_id = without_tracks.groupby(level=1)

# get a specific songs analysis
group_by_song_id.get_group('0UaMYEvWZi0ZqiDOoHU3YI')

duration  loudness  tempo  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0     1.95001       NaN    NaN   
                              1     1.89266       NaN    NaN   
                              2     1.89015       NaN    NaN   
                              3     1.91268       NaN    NaN   
                              4     1.91566       NaN    NaN   
...                                     ...       ...    ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941   0.23942       NaN    NaN   
                              942   0.24014       NaN    NaN   
                              943   0.24014       NaN    NaN   
                              944   0.24014       NaN    NaN   
                              945   0.24014       NaN    NaN   

                                   tempo_confidence  time_signature  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0                 NaN             NaN   
                              1                 NaN             NaN   
                              2                 NaN             NaN   
                              3                 NaN             NaN   
                              4                 NaN             NaN   
...                                             ...             ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941               NaN             NaN   
                              942               NaN             NaN   
                              943               NaN             NaN   
                              944               NaN             NaN   
                              945               NaN             NaN   

                                   time_signature_confidence  key  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0                          NaN  NaN   
                              1                          NaN  NaN   
                              2                          NaN  NaN   
                              3                          NaN  NaN   
                              4                          NaN  NaN   
...                                                      ...  ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941                        NaN  NaN   
                              942                        NaN  NaN   
                              943                        NaN  NaN   
                              944                        NaN  NaN   
                              945                        NaN  NaN   

                                   key_confidence  mode  mode_confidence  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0               NaN   NaN              NaN   
                              1               NaN   NaN              NaN   
                              2               NaN   NaN              NaN   
                              3               NaN   NaN              NaN   
                              4               NaN   NaN              NaN   
...                                           ...   ...              ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941             NaN   NaN              NaN   
                              942             NaN   NaN              NaN   
                              943             NaN   NaN              NaN   
                              944             NaN   NaN              NaN   
                              945             NaN   NaN              NaN   

                                       start  confidence  loudness_start  \
bars   0UaMYEvWZi0ZqiDOoHU3YI 0      0.24313       0.243             NaN   
                              1      2.19314       0.567             NaN   
                              2      4.08580       0.727             NaN   
                              3      5.97594       0.858             NaN   
                              4      7.88862       0.684             NaN   
...                                      ...         ...             ...   
tatums 0UaMYEvWZi0ZqiDOoHU3YI 941  225.26659       0.566             NaN   
                              942  225.50601       0.232       

### Data processed... now what?
